# Minimisation problems

Many problems in scientific computing (and other fields) boil down to solving a minimisation problems

$$
\min_{x} F(x; p)
$$

where $x \in \mathbb{R}^n$ are inputs we can control and $p \in \mathbb{R}^p$ are some fixed parameters.
We might also need to take into account constraints on the inputs $x$, but we'll brush that under the carpet here.

For differentiable $F$, minimisation problems are intimately linked to *rootfinding*. That is, we can rephrase the minimisation problem as a problem of finding $x$ such that

$$
G(x; p) := \nabla F(x; p) = 0
$$

In [ ]:
%matplotlib notebook
from matplotlib import pyplot
pyplot.style.use('ggplot')
import numpy

def g0(x):
    return x**2 - 2

## Used later
def dg0(x):
    return 2*x

x = numpy.linspace(-2,2,100)
pyplot.plot(x, g0(x), label="$G(x)$")
pyplot.plot(x, dg0(x), label="$G_x(x)$")
pyplot.legend(loc='upper right');

## Examples

### Neural networks

$F$ is the *loss function* which we are trying to minimise, $x$ are the weights in the network, and $p$ are the training data.

### Soap films

A soap film attached to a wire shape [minimises the surface area](https://en.wikipedia.org/wiki/Minimal_surface) of the film. Picking a coordinate system, and defining $u(x, y, z)$ as the height of the surface at each point in space we have

$$
G(u) = \nabla \cdot \left(\frac{\nabla u}{(1 + |\nabla u|^2)^{1/2}}\right) = 0
$$

### Stationary solutions of time-dependent equations

We have a time-dependent PDE

$$
\partial_t u - f(t, u) = 0,
$$

to solve for the steady state, we need to solve

$$
f(t, u) = 0.
$$

## Methods

Some questions immediately come to mind:

- **Existence**: does the equation have *at least one* solution?
- **Uniqueness**: if a solution exists, is the *only* solution?


### Bisection

With some mild restrictions on $G := \nabla F$ (that it is [continuous](https://en.wikipedia.org/wiki/Continuous_function)) we can exploit the intermediate value theorem to find a root.

Assume that $G$ changes sign on the interval $[a, b]$, then there must exist a $c \in [a, b]$ such that $G(c) = 0$.

In [ ]:
def bisect(f, a, b, tol=1e-5, history=None):
    midpoint = (a + b) / 2
    if b - a < tol:
        return midpoint, history
    elif abs(f(midpoint)) < tol:
        return midpoint, history
    if history is not None:
        history.append(midpoint)
    if f(a)*f(midpoint) < 0:
        return bisect(f, a, midpoint, tol=tol, history=history)
    else:
        return bisect(f, midpoint, b, tol=tol, history=history)
    
root, history = bisect(g0, -1, 2, history=[])

In [ ]:
root, numpy.sqrt(2)

In [ ]:
pyplot.figure()
pyplot.xlabel("iteration")
pyplot.ylabel("error")
pyplot.semilogy(numpy.abs(history - numpy.sqrt(2)), "o", label="Convergence")
pyplot.legend();

What about some other problems

In [ ]:
def g1(x):
    return numpy.exp(-numpy.abs(x)) + numpy.sin(x)

def dg1(x):
    return numpy.exp(-numpy.abs(x))*(-numpy.sign(x)) + numpy.cos(x)

x = numpy.linspace(-2,2,100)
pyplot.figure()
pyplot.plot(x, g1(x), label="$G(x)$")
pyplot.plot(x, dg1(x), label="$G_x(x)$")
pyplot.legend(loc='upper right');

In [ ]:
root, history = bisect(g1, -2.1, 2, history=[])

In [ ]:
root, g1(root)

In [ ]:
pyplot.figure()
x = numpy.linspace(-10,10,100)
pyplot.plot(x, g1(x))
pyplot.plot(history, g1(history), "o");

### Gradient descent

This is the workhorse of optimisation in machine learning algorithms. Suppose we want to minimise $F$ and have a way of evaluating $G$. Then given an *initial guess* $x_0$ we can produce a new guess for the minimum by walking "downhill" in the direction of the gradient:

$$
x_{i+1} \gets x_{i} - \alpha_i G(x_{i}).
$$

Where $\alpha_i$ is called the *learning rate* in the ML community, and a "linesearch damping" by everyone else. Informally, one chooses $\alpha_i$ so that we don't "overshoot".

In [ ]:
def gradient_descent(g, x, alpha, tol=1e-5, history=None, maxit=100):
    x = numpy.asarray(x)
    for i in range(maxit):
        gx = g(x)
        if numpy.linalg.norm(gx) < tol:
            return x, gx, i, history
        if history is not None:
            history.append(x.copy())
        x -= alpha*gx
    else:
        return x, gx, i, history
        raise RuntimeError(f"Didn't converge in {maxit} ({x, gx}) iterations")

In [ ]:
x0 = 1.
xstar, gx, i, history = gradient_descent(g1, x0, 0.1, tol=1e-10, maxit=1000, history=[])
xstar, g0(xstar)

In [ ]:
pyplot.figure()
pyplot.semilogy(numpy.abs(numpy.asarray(history) - history[-1]), ".-");

If we have a (local) minimum, with a sufficiently small damping parameter, gradient descent will find it eventually (but it might take a long time).

Gradient descent uses only first order information to compute roots and therefore converges only linearly to solutions. If it is feasible to compute second derivatives, then Newton's method can be used instead.

### Newton iteration

Suppose that $G$ is sufficiently smooth that we can form its first derivative, then we can write, given a guess $x_0$

$$
\tilde{G}(x) = G(x_0) + G'(x_0)(x - x_0) + \mathcal{O}((x - x_0)^2)
$$

Since $\tilde{G}$ is linear, we can explicitly solve for $\tilde{G}(x) = 0$

$$
x = x_0 - [G'(x_0)]^{-1}G(x_0)
$$

This update

$$
x_{i+1} \gets x_i - [G'(x_0)]^{-1} G(x_0)
$$

is a step in a [Newton iteration](https://en.wikipedia.org/wiki/Newton%27s_method).

Near a root (if the initial guess is good), this converges *quadratically* to the root. Similarly to gradient descent, it is often augmented with a line search to produce

$$
x_{i+1} \gets x_i - \alpha_i [G'(x_0)]^{-1} G(x_0).
$$

In [ ]:
def newton(g, dg, x, tol=1e-5, maxit=100, alpha=1, history=None):
    x = numpy.asarray(x)
    for i in range(maxit):
        gx = g(x)
        dgx = dg(x)
        if numpy.linalg.norm(gx) < tol:
            return x, gx, i, history
        if history is not None:
            history.append(x.copy())
        try:
            x -= alpha * numpy.linalg.inv(dgx) @ gx
        except numpy.linalg.LinAlgError:
            try:
                x -= alpha * gx/dgx
            except ZeroDivisionError:
                # exact root
                return x, numpy.NaN, i, history
    else:
        raise RuntimeError(f"Didn't converge after {maxit} iterations")

In [ ]:
x0 = 1.
xstar, residual, it, history = newton(g1, dg1, x0, alpha=0.1, tol=1e-10, maxit=1000, history=[])
xstar, g1(xstar)

In [ ]:
pyplot.figure()
pyplot.semilogy(numpy.abs(history - history[-1]), ".-");

### Observations

If the initial guess is not "close" in some sense to the root, then Newton's method only converges linearly (it is only near the root that quadratic convergence kicks in).

Notice how we didn't converge to the *same* root as gradient descent, even with the same initial guess. If gradient descent starts out in a valley of the function we're trying to minimise, it will never get out of it; Newton might.

In [ ]:
x = numpy.linspace(-10, 10, 200)
pyplot.figure()
pyplot.plot(x, g1(x));

## Finding global minima

In the general case, finding the global minimum of a high-dimensional function is an unsolvable problem. The best we can do is to try and find multiple local minima and hope that we found one that was "good enough". There are various techniques that are constructed on a somewhat more or less ad-hoc basis to do this.

If the function we're trying to minimise is [convex](https://en.wikipedia.org/wiki/Convex_function) then there are guarantees on existance, and uniqueness. Unfortunately, nature is not usually so kind.

## "Accelerated" gradient methods

In the ML community, computing the second derivative of the function to be minimised is generally computationally too expensive. While for a problem with $n$ parameters the gradient is a vector in $\mathbb{R}^n$, the second derivative is represented by a matrix in $\mathbb{R}^{n \times n}$. For mesh-based discretisations of PDEs this matrix is typically *a priori* sparse (as we have seen). In ML it may well not be.

Hence, there is quite a lot of interest in coming up with extensions of gradient descent that speed up convergence (especially near minima), hopefully attaining quadratic convergence in the best case.

One such example is *Nesterov acceleration*. At the cost of maintaining a little extra state, we can improve convergence in some cases.


In [ ]:
def nesterov(g, x, alpha=1, mu=0.8, tol=1e-5, maxit=100, history=None):
    yi = x
    v = 0
    for i in range(maxit):
        gx = g(x)
        if numpy.abs(gx) < tol:
            return x, gx, i, history
        # Standard gradient descent
        v = mu*v - alpha*gx
        x = x + mu*v - alpha*gx
        if history is not None:
            history.append(x)
    else:
        raise RuntimeError(f"Didn't converge in {maxit} iterations")

In [ ]:
x0 = 1.
xstar, residual, its, history = nesterov(g1, x0, alpha=0.5, mu=0.15, tol=1e-10, maxit=1000, history=[])
xstar, g1(xstar), its

In [ ]:
pyplot.figure()
pyplot.semilogy(numpy.abs(numpy.asarray(history) - history[-1]), ".-");

In [ ]:
x0 = 1.
xstar, residual, its, history = gradient_descent(g1, x0, alpha=0.5, tol=1e-10, maxit=1000, history=[])
xstar, g0(xstar), its

In [ ]:
pyplot.figure()
pyplot.semilogy(numpy.abs(numpy.asarray(history) - history[-1]), ".-");

### Some 2D problems

Let's look at a harder problem, that gradient descent really struggles with, but Newton works well.

This problem is very smooth but it has a very flat "valley" with a minimum in it. Near the minimum, therefore, gradient descent really slows down, while far away we need a good damping parameter to not diverge.

In [ ]:
from mpl_toolkits import mplot3d
def f(x, y):
    return (1 - x)**2 + 100*(y-x**2)**2

x = numpy.linspace(-2, 2, 100)
x, y = numpy.meshgrid(x, x)
pyplot.figure()
ax = pyplot.axes(projection='3d')

ax.plot_surface(x, y, f(x, y), cmap="viridis");

In [ ]:
# For gradient descent
def g(X):
    x, y = X
    return numpy.asarray((2*(200*x**3 - 200*x*y + x - 1), 200*(y - x**2)))

# For newton
def dg(X):
    x, y = X
    return numpy.asarray([[1200*x**2 - 400*y + 2, -400*x],
                          [-400*x, 200]])

In [ ]:
x0 = numpy.asarray([0.999, 0.999], dtype=float)

xstar, gx, its, history = gradient_descent(g, x0, 0.0025, tol=1e-5, maxit=10000, history=[])
xstar, f(*xstar), gx, its

In [ ]:
pyplot.figure()
x = numpy.linspace(-2, 2, 100)
x, y = numpy.meshgrid(x, x)
pyplot.contourf(x, y, f(x, y), levels=20)
pyplot.colorbar()
pyplot.plot(*zip(*history), ".-");

In [ ]:
x0 = numpy.asarray([0.25, 0.25], dtype=float)

history = []
xstar, gx, its, history = newton(g, dg, x0, alpha=1, tol=1e-10, maxit=1000, history=history)
xstar, f(*xstar), its

In [ ]:
pyplot.figure()
x = numpy.linspace(-2, 2, 100)
x, y = numpy.meshgrid(x, x)
pyplot.contourf(x, y, f(x, y), levels=20)
pyplot.colorbar()
pyplot.plot(*zip(*history), ".-");

In [ ]:
f(1.001, 1)